In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.express as px

# Exploration des communes

On cherche ici surtout à obtenir la population, le nom et le code INSEE des communes, afin d'ultérieurement faire un merging avec la liste des gares et leur fréquentation (pour voir quelles gares ont une fréquentation en hausse par rapport à la population de la ville dans laquelle elles se trouvent).

```communes-france.csv``` et ```insee-pop-communes.csv```

## ```insee-pop-communes.csv```

In [2]:
population = pd.read_csv("../data/insee-pop-communes.csv",sep=";")
print(population.shape)
population.head()

(34995, 5)


,DEPCOM,COM,PMUN,PCAP,PTOT
0,01001,L' Abergement-Clémenciat,776,18,794
1,01002,L' Abergement-de-Varey,248,1,249
2,01004,Ambérieu-en-Bugey,14035,393,14428
3,01005,Ambérieux-en-Dombes,1689,34,1723
4,01006,Ambléon,111,6,117


On n'a besoin que de la population totale, donc on supprimera ```PMUN``` et ```PCAP```.

On supprime également ```COM``` car le nom de la commune est aussi présent dans ```communes-france.csv```.

## ```communes-france.csv```

In [3]:
communes = pd.read_csv("../data/communes-france.csv",sep=";")
print(communes.shape)
print(communes.columns)
communes.head()

(2601, 40)
Index(['Geo Point', 'Geo Shape', 'Année', 'Code Officiel Région',
       'Nom Officiel Région', 'Code Officiel Département',
       'Nom Officiel Département',
       'Code Officiel Arrondissement départemental',
       'Nom Officiel Arrondissement départemental',
       'Code Officiel Zone emploi 2020', 'Nom Officiel Zone emploi 2020',
       'Code Officiel Bassin vie 2022', 'Nom Officiel Bassin vie 2022',
       'Code Officiel EPCI', 'Nom Officiel EPCI', 'Code Officiel EPT',
       'Nom Officiel EPT', 'Code Officiel Commune',
       'Code Officiel Courant Commune', 'Nom Officiel Commune',
       'Nom Officiel Commune Majuscule', 'Nom Officiel Commune Minuscule',
       'Code Iso 3166-3 Zone', 'Type', 'Nom Officiel Zone emploi 2010',
       'Code Officiel Zone emploi 2010', 'Code CATAEU2010',
       'Libellé CATAEU2010', 'Code UU2010', 'Statut Commune UU2010',
       'Code AU2010', 'Code CATEAAV2020', 'Libellé CATEAAV2020', 'Code UU2020',
       'Statut Commune UU2020', 'Co

,Geo Point,Geo Shape,Année,Code Officiel Région,Nom Officiel Région,Code Officiel Département,Nom Officiel Département,Code Officiel Arrondissement départemental,Nom Officiel Arrondissement départemental,Code Officiel Zone emploi 2020,...,Code AU2010,Code CATEAAV2020,Libellé CATEAAV2020,Code UU2020,Statut Commune UU2020,Code AAV2020,Code Canton Ville,Fait Partie d'une CTU,SIREN,Zone Montagne
0,"45.445282333838946, 2.063226732075124","{""coordinates"": [[[2.0969856744729, 45.4477906...",2024,75,Nouvelle-Aquitaine,19,Corrèze,193,Ussel,63.0,...,NaN,20.0,Commune de la couronne,19000,hors unité urbaine,499,1912,Non,NaN,Oui
1,"46.98785031385113, 4.596669285873805","{""coordinates"": [[[4.564662411513994, 46.99215...",2024,27,Bourgogne-Franche-Comté,21,Côte-d'Or,211,Beaune,2704.0,...,NaN,30.0,Commune hors attraction des villes,21000,hors unité urbaine,000,2101,Non,NaN,Non
2,"47.43924794029317, 5.355638135542026","{""coordinates"": [[[5.379160624539277, 47.45041...",2024,27,Bourgogne-Franche-Comté,21,Côte-d'Or,212,Dijon,2711.0,...,NaN,20.0,Commune de la couronne,21000,hors unité urbaine,028,2121,Non,NaN,Non
3,"47.23884419720391, 4.9564962791807865","{""coordinates"": [[[5.025355389789206, 47.23519...",2024,27,Bourgogne-Franche-Comté,21,Côte-d'Or,211,Beaune,2711.0,...,NaN,20.0,Commune de la couronne,21110,unité urbaine,028,2118,Non,NaN,Non
4,"47.356304694101055, 4.7137360827879835","{""coordinates"": [[[4.743403179150424, 47.34145...",2024,27,Bourgogne-Franche-Comté,21,Côte-d'Or,212,Dijon,2711.0,...,NaN,20.0,Commune de la couronne,21000,hors unité urbaine,028,2123,Non,NaN,Non


In [4]:
def treat_and_merge_data(communes_df,population_df):
    # Communes
    communes_relevant_columns = ['Nom Officiel Région', 'Code Officiel Département', 'Nom Officiel Région', 'Nom Officiel Département', 'Code Officiel Commune', 'Nom Officiel Commune', "Nom Officiel Arrondissement départemental", 'Statut Commune UU2020']
    communes_treated = communes.query("Année == 2024")
    communes_treated = communes_treated[communes_relevant_columns]
    communes_treated = communes_treated.rename(columns={'Nom Officiel Région':'Région','Code Officiel Département':'Code Département','Nom Officiel Région':'Région','Nom Officiel Département':'Département','Nom Officiel Commune':'Commune','Nom Officiel Arrondissement départemental':'Aire Urbaine','Statut Commune UU2020':'Statut Commune'})
    # Population
    population_relervant_columns = ['DEPCOM', 'PTOT']
    population_treated = population[population_relervant_columns]
    population_treated = population_treated.rename(columns={'DEPCOM':'Code Officiel Commune'})
    
    merged_df = pd.merge(communes_treated,population_treated,on='Code Officiel Commune')
    
    return merged_df

communes_population = treat_and_merge_data(communes,population)
print(communes_population.shape)
communes_population.head()    

(2597, 9)


,Région,Code Département,Région,Département,Code Officiel Commune,Commune,Aire Urbaine,Statut Commune,PTOT
0,Nouvelle-Aquitaine,19,Nouvelle-Aquitaine,Corrèze,19263,Soudeilles,Ussel,hors unité urbaine,312
1,Bourgogne-Franche-Comté,21,Bourgogne-Franche-Comté,Côte-d'Or,21032,Aubigny-la-Ronce,Beaune,hors unité urbaine,171
2,Bourgogne-Franche-Comté,21,Bourgogne-Franche-Comté,Côte-d'Or,21079,Blagny-sur-Vingeanne,Dijon,hors unité urbaine,132
3,Bourgogne-Franche-Comté,21,Bourgogne-Franche-Comté,Côte-d'Or,21110,Brochon,Beaune,unité urbaine,776
4,Bourgogne-Franche-Comté,21,Bourgogne-Franche-Comté,Côte-d'Or,21121,Bussy-la-Pesle,Dijon,hors unité urbaine,78
